In [1]:
import sqlite3
import numpy as np
import pandas as pd
import ctypes
import shutil
import os
MessageBox = ctypes.windll.user32.MessageBoxW
from System_Assessment_Generate_Models_Variables import *


In [2]:
def sql_to_df(sql,model):
    con = sqlite3.connect(model)
    df = pd.read_sql(sql, con)
    con.close()
    return df

def execute_sql(sqls,model):
    con = sqlite3.connect(model)
    cur = con.cursor()
    if type(sqls) == list:
        for sql in sqls:
            print(sql)
            cur.execute(sql)
    else:
        sql = sqls
        cur.execute(sql)
    cur.close()
    con.commit()
    con.close()
  
def df_to_sql(df,table_name,model):
    conn = sqlite3.connect(model)
    df.to_sql(table_name, conn, if_exists='replace', index=False)
    conn.commit()
    conn.close()
    
def generate_script(script_path,mu_path):  
    script_path_new = os.path.splitext(mu_path)[0] + '.cs'
    shutil.copy(script_path, script_path_new)
    
def generate_mupp(model_original,mu_path):
    mupp_path_original = os.path.splitext(model_original)[0] + '.mupp'
    mupp_path_new = os.path.splitext(mu_path)[0] + '.mupp'   
    with open(mupp_path_original, 'r') as source_file, open(mupp_path_new, 'w') as output_file:
        for line in source_file:
            if line.startswith(r'   DBFilePath = |.'):
                output_file.write('   DBFilePath = |.\\' + os.path.splitext(model_name)[0] + '.sqlite|\n')
            else:
                output_file.write(line)
    

In [3]:
if generate_future:
    
    mu_paths = []

    keep_cols = ['Catchment','Zone','Year','Pop_ResLD','Pop_ResHD','Pop_Mixed','Area_Com','Area_Ind','Area_Inst']
    pop_df = pd.read_excel(population_sheet,sheet_name=population_tab,usecols=keep_cols,dtype={'Catchment': str})

    # pop_df = pop_df[pop_df.Catchment!='84394S']
    # pop_df = pop_df[pop_df.Catchment!='84279']
    # pop_df = pop_df.drop(pop_df.index[-1])

    # Melt the DataFrame to combine the columns
    columns_to_combine = ['Pop_ResLD', 'Pop_ResHD', 'Pop_Mixed', 'Area_Com', 'Area_Ind', 'Area_Inst']
    pop_df_melt = pd.melt(pop_df, id_vars=['Catchment', 'Zone', 'Year'], value_vars=columns_to_combine, var_name='Type', value_name='Value')
    pop_df_melt[['Major_type', 'Minor_type']] = pop_df_melt['Type'].str.split('_', n=1, expand=True)
    pop_df_melt = pop_df_melt.drop(columns=['Type'])
    pop_df_melt['MUID'] = pop_df_melt.Catchment + '_' + pop_df_melt.Minor_type
    pop_df_melt

    #Check for missing catchments
    catchment_years = []
    sql = "SELECT catchmentid FROM msm_Loadpoint GROUP BY catchmentid"
    muids = list(sql_to_df(sql,model_original).catchmentid)

    for muid in muids:
        for year_scenario in year_scenario_list:
            year = year_scenario[0]
            catchment_years.append([muid,year])
    catchment_year_df = pd.DataFrame(catchment_years,columns=(['Catchment','Year']))
    catchment_year_df 

    merged = catchment_year_df.merge(pop_df[['Catchment', 'Year']], on=['Catchment', 'Year'], how='left', indicator=True)

    not_founds = merged[merged['_merge'] == 'left_only'].drop(columns=['_merge'])

    if len(not_founds) > 0:
        message = "WARNING.The following catchment/year combinations are not found\n\n"
        for index, row in not_founds.iterrows():
            message += row[0] + ', ' + str(row[1]) + '.\n'
        message += '\nContinue?'

        if MessageBox(None, message, 'Warning', 4) == 7:
            MessageBox(None, "Please report the missing catchment(s)", 'Info', 0)
            raise ValueError(message)
        else:
            pass

    for year_scenario in year_scenario_list:
        year = year_scenario[0]
        scenario = year_scenario[1]
        turnons = year_scenario[2]

        model_name = model_area + '_' + str(year) + 'pop_V' + str(version) + '.sqlite'

        if os.path.basename(model_original) == model_name:

            message = "Tool ends. For year " + str(year) + ", the new model name '" + model_name + "' is the same as the original."
            MessageBox(None, message, 'Info', 0)
            raise ValueError("message")

        #Delete sqlite, mupp and cs if they exist and create new
        mu_path = output_folder + "\\" + model_name
        
        mu_paths.append(mu_path)

        generate_script(script_path,mu_path)
        generate_mupp(model_original,mu_path)

        os.remove(mu_path) if os.path.exists(mu_path) else None
        shutil.copyfile(model_original, mu_path)

        sql = "SELECT MUID FROM msm_Project WHERE enable_hd = 1"
        muids = list(sql_to_df(sql,mu_path).muid)
        for muid in muids:
            muid_new = muid.replace(str(year_original) + 'p',str(year) + 'p')
            sql = "UPDATE msm_Project SET scenarioname = '" + scenario + "', muid = '" + muid_new + "' WHERE muid = '" + muid + "'"
            execute_sql(sql, mu_path)

            sql = "UPDATE msm_ProjectOutput SET simulationid = '" + muid_new + "' WHERE simulationid = '" + muid + "'"
            execute_sql(sql, mu_path)

        sql = "UPDATE msm_Project SET IncludeToBatchNo = 0 WHERE MUID LIKE '%h-AES_%'"
        execute_sql(sql, mu_path)

        for turnon in turnons:
            sql = "UPDATE msm_Project SET IncludeToBatchNo = 1 WHERE MUID LIKE '%h-AES_%' AND MUID LIKE '%" + turnon + "%'"
            execute_sql(sql, mu_path)

        pop_df_melt_year = pop_df_melt[pop_df_melt.Year==year]

        df_to_sql(pop_df_melt_year,'New_Population',mu_path)

        sql = "UPDATE msm_Loadpoint SET Population = "
        sql += "(SELECT Value FROM New_Population WHERE MUID = msm_Loadpoint.muid AND Major_Type = 'Pop')"
        execute_sql(sql, mu_path)

        sql = "UPDATE msm_Loadpoint SET ICIArea = "
        sql += "(SELECT Value FROM New_Population WHERE MUID = msm_Loadpoint.muid AND Major_Type = 'Area')"
        execute_sql(sql, mu_path)

        sql = "UPDATE msm_Loadpoint SET loadflow = PerCapitaLoad * Population / 86400 WHERE LoadCategory = 'Mixed' OR LoadCategory = 'ResLD' OR LoadCategory = 'ResHD'"
        execute_sql(sql, mu_path)
        sql = "UPDATE msm_Loadpoint SET loadflow = PerAreaLoad * ICIArea / 86400 WHERE LoadCategory = 'Commercial' OR LoadCategory = 'Industrial' OR LoadCategory = 'Institutional'"
        execute_sql(sql, mu_path)

        sql = "DROP TABLE New_Population"
        execute_sql(sql, mu_path)

    

In [4]:
##AFTER UPDATE AND SAVE YOU MUST RESTART THE KERNEL IN JUPYTER NOTEBOOK TO UPDATE VARIABLES!

##Remember to insert r in front of all paths, e.g. r"J:\SEWER_AREA_MODELS\FSA\03_SIMULATION_WORK\Calibration_2022\MODEL"


#FSA System Assessment

generate_future = False
generate_sealed_vfd = True
generate_xadwf = False
generate_bsf = False

output_folder = r'J:\SEWER_AREA_MODELS\FSA\03_SIMULATION_WORK\System_Assessment\Model_Generation'

#For future models
model_original = r"J:\SEWER_AREA_MODELS\FSA\01_MASTER_MODEL\MODEL\FSA_Base_2021pop.sqlite"
year_original = 2021
version = 103
year_scenario_list = []
year_scenario_list.append([2030,'2030_Network',['Ex']])
year_scenario_list.append([2040,'2030_Network',['2050M','2050H']])
year_scenario_list.append([2050,'2030_Network',['2050H']])
year_scenario_list.append([2060,'2030_Network',['2050H','2100H']])
population_sheet = r"J:\SEWER_AREA_MODELS\FSA\02_MODEL_COMPONENTS\04_DATA\01. POPULATION\FSA_Master_Population_File_Update14a15a.xlsx"
population_tab = 'MPF Update 14a'

#For sealed VFD
vfd_all = True
seal_all = True

excluded_asset_names = []
excluded_asset_names.append('Golden Ears SSO Tank')
excluded_asset_names.append('NW CSO Tank Cleanout Pumps')
excluded_asset_names.append('')

weir_turn_offs = []
weir_turn_offs.append('NS4-SSO')


In [5]:
seal_all and vfd_all

True

In [6]:
if generate_sealed_vfd:

    if not generate_future:
        mu_paths = []
        for f in os.listdir(output_folder):
            if f[-7:]==".sqlite" and not 'VFD' in f:
                mu_paths.append(output_folder + '\\' + f)
 
    if seal_all and vfd_all:
        suffix = "S_V_"
        file_suffix = "_Sealed_VFD"
    elif seal_all and not vfd_all:
        suffix = "S_"
        file_suffix = "_Sealed"
    elif not seal_all and vfd_all:        
        suffix = "V_"
        file_suffix = "_VFD" 
    else:
        message = "Error! generate_sealed_vfd set to True but both vfd_all and seal_all set to False"
        raise ValueError(message)

    for mu_path_original in mu_paths[2:]:
        
        

        sqls = []

        mu_path = mu_path_original[:-7] + file_suffix + ".sqlite"
        os.remove(mu_path) if os.path.exists(mu_path) else None
        shutil.copyfile(mu_path_original, mu_path)
        
        generate_script(script_path,mu_path)
        generate_mupp(mu_path_original,mu_path)
        
        print('Generating ' + mu_path)

        if vfd_all == True:

            pumps_for_VFD = []
            pumps_turn_off = []

            sqls.append("INSERT INTO ms_Tab (muid,altid,active,description,typeno) SELECT 'Generic_Pump_Min',0,1,'Use for PS passing all inflow',2")
            sqls.append("INSERT INTO ms_Tab (muid,altid,active,description,typeno) SELECT 'Generic_Pump_Max',0,1,'Use for PS passing all inflow',2")

            sqls.append("INSERT INTO ms_TabD (muid,altid,active,tabid,sqn,value1,value2) SELECT 'Generic_Pump_Max-1',0,1,'Generic_Pump_Max',1, 0, 50")
            sqls.append("INSERT INTO ms_TabD (muid,altid,active,tabid,sqn,value1,value2) SELECT 'Generic_Pump_Max-2',0,1,'Generic_Pump_Max',2, 100, 50")
            
            sqls.append("INSERT INTO ms_TabD (muid,altid,active,tabid,sqn,value1,value2) SELECT 'Generic_Pump_Min-1',0,1,'Generic_Pump_Min',1, 0, 0")
            sqls.append("INSERT INTO ms_TabD (muid,altid,active,tabid,sqn,value1,value2) SELECT 'Generic_Pump_Min-2',0,1,'Generic_Pump_Min',2, 100, 0")

            
            sql = "SELECT MUID FROM msm_Project WHERE enable_hd = 1"
            muids = list(sql_to_df(sql,mu_path).muid)
            for muid in muids:
                if len(muid) <= suffix:
                    muid_new = muid & suffix
                    sqls.append("UPDATE msm_Project SET muid = '" + muid_new + "'' WHERE muid = '" + muid + "'")        
                    sqls.append("UPDATE msm_ProjectOutput SET simulationid = '" + muid_new + "' WHERE simulationid = '" + muid + "'"
            sql = "SELECT assetname, muid, startlevel from msm_Pump ORDER BY assetname, startlevel"
            pumps = sql_to_df(sql,mu_path)

            previous_asset = 'xxxx'
            for index, row in pumps.iterrows():
                asset = str(row[0])
                if not asset in excluded_asset_names and asset != 'None':
                    muid = str(row[1])
                    if previous_asset != asset:
                        pumps_for_VFD.append(muid)
                    else:
                        pumps_turn_off.append (muid)
                    previous_asset = asset
                else:
                    print ('Skipped ' + str(row[1]))

            for pump_for_VFD in pumps_for_VFD:
                sqls.append("UPDATE msm_Pump SET qmaxsetid = 'Generic_Pump_Max',  qminsetid = 'Generic_Pump_Min', speedno = 2, captypeno = 2, controltypeno = 1, startlevel = stoplevel + 0.1,  wetwellsetpoint = stoplevel + 0.1  WHERE muid = '" + pump_for_VFD + "'")
                sqls.append("UPDATE msm_RTC SET applyno = 0 WHERE pumpid = '" + pump_for_VFD + "'")

            for pump_turn_off in pumps_turn_off:
                sqls.append("UPDATE msm_Pump SET startlevel = startlevel + 100, stoplevel = stoplevel + 100,  controltypeno = 1 WHERE MUID = '" + pump_turn_off + "'")
                sqls.append("UPDATE msm_RTC SET applyno = 0 WHERE pumpid = '" + pump_turn_off + "'")


        execute_sql(sqls, mu_path)


SyntaxError: invalid syntax (3499044034.py, line 59)